In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

/Users/oanaalexandrablanc/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/oanaalexandrablanc/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
# Specify the file path
file_path_test = "unlabelled_test_data.csv"
file_path_training = "training_data.csv"

# Read the CSV file
unlabelled_test_data = pd.read_csv(file_path_test)
training_data = pd.read_csv(file_path_training)

In [3]:
import string
import spacy
# Load the French spaCy model

nlp = spacy.load("fr_core_news_sm")

def preprocess_text(text):

    """

    Preprocess the text by:

    - Lowercasing

    - Removing punctuation

    - Lemmatizing (using French spaCy model)

    """

    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Lemmatization using spaCy
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])

    return lemmatized_text

# Apply the preprocessing to the 'sentence' column

training_data['processed_sentence'] = training_data['sentence'].apply(preprocess_text)

# Display the first few rows of the updated dataframe
training_data.head()

,id,sentence,difficulty,processed_sentence
0,0,Les coûts kilométriques réels peuvent diverger...,C1,le coût kilométrique réel pouvoir diverger sen...
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1,le bleu cest mon couleur préférer mais je naim...
2,2,Le test de niveau en français est sur le site ...,A1,le test de niveau en français être sur le site...
3,3,Est-ce que ton mari est aussi de Boston?,A1,estce que ton mari être aussi de boston
4,4,"Dans les écoles de commerce, dans les couloirs...",B1,dans le école de commerce dans le couloir de p...


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(training_data['sentence'])
y = training_data['difficulty']

In [5]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [6]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [10]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Assuming y_train is your target variable in the training dataset
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Map the computed weights to the corresponding class labels
class_weights_dict = dict(zip(np.unique(y_train), class_weights))

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))

# Train a Random Forest model
random_forest_model = RandomForestClassifier(n_estimators=100, class_weight=class_weights_dict)
random_forest_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_rf = random_forest_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Random Forest Accuracy: {accuracy_rf}')

Random Forest Accuracy: 0.37083333333333335
